In [1]:
%matplotlib nbagg

In [2]:
import os
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from patsy import dmatrices
import matplotlib.pyplot as plt

In [3]:
DATADIR = os.path.join(os.path.expanduser("~"), "DATA", "TimeSeries", "EPA")
os.path.exists(DATADIR)

True

In [4]:
files = os.listdir(DATADIR)
files

['aqs_data_dictionary.pdf',
 'San_Diego_2016_PM25.xlsx',
 'Salt_Lake_2016_PM25.xlsx',
 'Asthma_SLC.xlsx',
 'SLC_Weather_2016.xlsx']

## Read the air quality data

In [5]:
slc = pd.read_excel(os.path.join(DATADIR, 'Salt_Lake_2016_PM25.xlsx'))
print(slc.columns)
slc.head()

Index(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC',
       'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Date Local',
       'Time Local', 'Date GMT', 'Time GMT', 'Sample Measurement',
       'Units of Measure', 'MDL', 'Uncertainty', 'Qualifier', 'Method Type',
       'Method Code', 'Method Name', 'State Name', 'County Name',
       'Date of Last Change'],
      dtype='object')


,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Date Local,...,Units of Measure,MDL,Uncertainty,Qualifier,Method Type,Method Code,Method Name,State Name,County Name,Date of Last Change
0,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
1,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
2,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
3,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
4,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22


In [6]:
i = 2
slc.iloc[i:i+10,10:15]["Sample Measurement"].mean()

17.23

In [ ]:
d1 = slc["Date Local"][0]
t1 = slc["Time Local"][0]

In [13]:
t1

datetime.time(13, 0)

In [11]:
print("Date Local type",type(d1),"Time Local type",type(t1))
datetime.datetime.combine(d1.to_pydatetime(), t1)

Date Local type <class 'pandas._libs.tslibs.timestamps.Timestamp'> Time Local type <class 'datetime.time'>


datetime.datetime(2016, 1, 7, 13, 0)

### Comments:

* Dates and times are split into separate columns
* We have both local time and UTC time 

## Merging Dates and Time
* After we have read in the data we could combine the dates and times
    * Convert pandas Timestamp to datetime
    * Use ``datetime.combine`` to merge with ``datetime.time``
* We can also merge these when reading the data by specifying the ``parse_dates`` keyword

#### Using ``datetime.combine``

In [14]:
slc.apply(lambda x: datetime.datetime.combine(x["Date Local"],x["Time Local"]), axis=1).head()

0   2016-01-07 13:00:00
1   2016-01-07 14:00:00
2   2016-01-07 15:00:00
3   2016-01-07 16:00:00
4   2016-01-07 17:00:00
dtype: datetime64[ns]

In [15]:
sd = pd.read_excel(os.path.join(DATADIR, "San_Diego_2016_PM25.xlsx"))
sd.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Date Local,...,Units of Measure,MDL,Uncertainty,Qualifier,Method Type,Method Code,Method Name,State Name,County Name,Date of Last Change
0,6,73,1011,88101,3,32.725226,-116.365203,WGS84,PM2.5 - Local Conditions,2016-01-01,...,Micrograms/cubic meter (LC),5,NaN,NaN,FEM,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,California,San Diego,2016-04-12
1,6,73,1011,88101,3,32.725226,-116.365203,WGS84,PM2.5 - Local Conditions,2016-01-01,...,Micrograms/cubic meter (LC),5,NaN,NaN,FEM,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,California,San Diego,2016-04-12
2,6,73,1011,88101,3,32.725226,-116.365203,WGS84,PM2.5 - Local Conditions,2016-01-01,...,Micrograms/cubic meter (LC),5,NaN,NaN,FEM,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,California,San Diego,2016-04-12
3,6,73,1011,88101,3,32.725226,-116.365203,WGS84,PM2.5 - Local Conditions,2016-01-01,...,Micrograms/cubic meter (LC),5,NaN,NaN,FEM,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,California,San Diego,2016-04-12
4,6,73,1011,88101,3,32.725226,-116.365203,WGS84,PM2.5 - Local Conditions,2016-01-01,...,Micrograms/cubic meter (LC),5,NaN,NaN,FEM,170,Met One BAM-1020 Mass Monitor w/VSCC - Beta At...,California,San Diego,2016-04-12


## Read in weather data

In [16]:
slc_weather = pd.read_excel(os.path.join(DATADIR, 'SLC_Weather_2016.xlsx'), skiprows=[1])
slc_weather.head()

,Day,High,Low,Precip.,Snow,Snow Depth
0,2016-01-01,21.9,9,0,0,0
1,2016-01-02,23,10.9,0,0,0
2,2016-01-03,27,21,0,0,0
3,2016-01-04,32,21.9,0,0,0
4,2016-01-05,45,28,0,0,0


In [17]:
slc_weather.columns

Index(['Day', 'High', 'Low', 'Precip.', 'Snow', 'Snow Depth'], dtype='object')

In [18]:
slc_weather['Day'][0]

Timestamp('2016-01-01 00:00:00')

## Our Weather has Resolution of Days
## Our pollutant data has resolution of hours
## Group and take mean?

In [19]:
slc.groupby("Date Local", as_index=False).aggregate(np.mean)

,Date Local,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Sample Measurement,MDL,Uncertainty,Method Code
0,2016-01-01,49.0,35.0,3006.000000,88101.0,5.000000,40.736389,-111.872222,19.208333,2.0,NaN,182.000000
1,2016-01-02,49.0,35.0,3006.000000,88101.0,5.000000,40.736389,-111.872222,23.558333,2.0,NaN,182.000000
2,2016-01-03,49.0,35.0,3006.000000,88101.0,5.000000,40.736389,-111.872222,26.504167,2.0,NaN,182.000000
3,2016-01-04,49.0,35.0,3006.000000,88101.0,5.000000,40.736389,-111.872222,34.120833,2.0,NaN,182.000000
4,2016-01-05,49.0,35.0,3006.000000,88101.0,5.000000,40.736389,-111.872222,25.291667,2.0,NaN,182.000000
5,2016-01-06,49.0,35.0,3006.000000,88101.0,5.000000,40.736389,-111.872222,13.616667,2.0,NaN,182.000000
6,2016-01-07,49.0,35.0,3006.000000,88101.0,4.685714,40.736389,-111.872222,14.891429,2.0,NaN,182.628571
7,2016-01-08,49.0,35.0,3006.000000,88101.0,4.500000,40.736389,-111.872222,5.047917,2.0,NaN,183.000000
8,2016-01-09,49.0,35.0,3006.000000,88101.0,4.500000,40.736389,-111.872222,7.997917,2.0,NaN,183.000000
9,2016-01-10,49.0,35.0,3006.000000,88101.0,4.500000,40.736389,-111.872222,8.222917,2.0,NaN,183.000000


### Group and take sum?

In [20]:
slc.groupby("Date Local", as_index=False).aggregate(np.sum)

,Date Local,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Sample Measurement,MDL,Uncertainty,Method Code
0,2016-01-01,1176,840,72144,2114424,120,977.673336,-2684.933328,461.0,48,0.0,4368
1,2016-01-02,1176,840,72144,2114424,120,977.673336,-2684.933328,565.4,48,0.0,4368
2,2016-01-03,1176,840,72144,2114424,120,977.673336,-2684.933328,636.1,48,0.0,4368
3,2016-01-04,1176,840,72144,2114424,120,977.673336,-2684.933328,818.9,48,0.0,4368
4,2016-01-05,1176,840,72144,2114424,120,977.673336,-2684.933328,607.0,48,0.0,4368
5,2016-01-06,1176,840,72144,2114424,120,977.673336,-2684.933328,326.8,48,0.0,4368
6,2016-01-07,1715,1225,105210,3083535,164,1425.773615,-3915.527770,521.2,70,0.0,6392
7,2016-01-08,2352,1680,144288,4228848,216,1955.346672,-5369.866656,242.3,96,0.0,8784
8,2016-01-09,2352,1680,144288,4228848,216,1955.346672,-5369.866656,383.9,96,0.0,8784
9,2016-01-10,2352,1680,144288,4228848,216,1955.346672,-5369.866656,394.7,96,0.0,8784


### Applying different functions to different columns

In [21]:
slc_day = slc.groupby(["Date Local", "Site Num"], 
                      as_index=False).aggregate(
                         {'Sample Measurement' : np.mean,
                          'MDL': np.median})
slc_day.head()

,Date Local,Site Num,Sample Measurement,MDL
0,2016-01-01,3006,19.208333,2
1,2016-01-02,3006,23.558333,2
2,2016-01-03,3006,26.504167,2
3,2016-01-04,3006,34.120833,2
4,2016-01-05,3006,25.291667,2


In [22]:
slc_day_all = slc_day.merge(slc_weather, 
                            left_on="Date Local", 
                            right_on="Day")
slc_day_all.head()

,Date Local,Site Num,Sample Measurement,MDL,Day,High,Low,Precip.,Snow,Snow Depth
0,2016-01-01,3006,19.208333,2,2016-01-01,21.9,9,0,0,0
1,2016-01-02,3006,23.558333,2,2016-01-02,23,10.9,0,0,0
2,2016-01-03,3006,26.504167,2,2016-01-03,27,21,0,0,0
3,2016-01-04,3006,34.120833,2,2016-01-04,32,21.9,0,0,0
4,2016-01-05,3006,25.291667,2,2016-01-05,45,28,0,0,0


### Rename columns to eliminate spaces

In [23]:
{c:c.replace(" ", "_") for c in slc_day_all.columns}

{'Date Local': 'Date_Local',
 'Site Num': 'Site_Num',
 'Sample Measurement': 'Sample_Measurement',
 'MDL': 'MDL',
 'Day': 'Day',
 'High': 'High',
 'Low': 'Low',
 'Precip.': 'Precip.',
 'Snow': 'Snow',
 'Snow Depth': 'Snow_Depth'}

In [24]:
slc_final = slc_day_all.rename(
    columns={c:c.replace(" ", "_") for c in slc_day_all.columns}
)

### What would be an interesting plot?